In [6]:
!python -m wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py


Saved under minsearch.py


In [8]:
import minsearch

In [9]:
import json

with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [11]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [12]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [15]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [ ]:
SELECT  * WHERE course = 'data-engineering-zoomcamp';


In [13]:
q = 'the course has already started, can I still enroll?'

In [16]:
index.fit(documents)

In [20]:
boost ={'question': 3.0, 'section':0.5}

results = index.search(
    query=q,
    filter_dict={'course':'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5
)
    

In [21]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [22]:
from openai  import OpenAI

In [31]:
client = OpenAI()

In [35]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role":"user","content":q}]
)

response

In [40]:
response.choices[0].message.content

"If the course has already started, whether you can still enroll depends on several factors including the institution's policies, the specific course, and how much time has elapsed since the start date. Here are some steps you can take:\n\n1. **Check the Enrollment Policy:** Look for the enrollment or add/drop policy on the institution’s website. Some institutions allow late enrollment within a certain timeframe.\n\n2. **Contact the Instructor:** Reach out to the course instructor to see if they are willing to allow you to join late. Explain your situation and express your commitment to catching up.\n\n3. **Administrative Offices:** Contact the registrar's office or student services. They can provide information on whether late enrollment is possible and what steps you need to take.\n\n4. **Self-Assessment:** Consider your ability to catch up on missed lectures, assignments, or exams. Some courses may be more flexible than others when it comes to catching up.\n\n5. **Financial Aid:** I

In [47]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [48]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [49]:
prompt = prompt_template.format(question=q, context=context).strip()

In [51]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role":"user","content":prompt}]
)



ChatCompletion(id='chatcmpl-9tHoSYbkbMDyhttjmoQqFrhxzDmLE', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Yes, you can still enroll in the course after it has started. You are eligible to submit the homeworks, but be mindful of the deadlines for turning in the final projects. So, don’t leave everything for the last minute.', role='assistant', function_call=None, tool_calls=None))], created=1722963540, model='gpt-4o-2024-05-13', object='chat.completion', service_tier=None, system_fingerprint='fp_3aa7262c27', usage=CompletionUsage(completion_tokens=47, prompt_tokens=514, total_tokens=561))

In [52]:
response.choices[0].message.content

'Yes, you can still enroll in the course after it has started. You are eligible to submit the homeworks, but be mindful of the deadlines for turning in the final projects. So, don’t leave everything for the last minute.'

In [80]:
def search(query):
    boost ={'question': 3.0, 'section':0.5} #le da más peso a si hay una coincidencia de palabras entre el query y la pregunta.
    
    results = index.search(
        query=query,
        filter_dict={'course':'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=10
    )

    return results
    

In [81]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    If the CONTEXT doesn't contain the answer, output NONE.
    
    QUESTION: {question}
    
    CONTEXT:
    {context}
    """.strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()

    return prompt
    

In [82]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role":"user","content":prompt}]
    )
    
    return response.choices[0].message.content

In [87]:
query = 'how do i run kafka?'

def rag(query):
    
    search_results= search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)

    return answer

In [88]:
rag(query)

'To run Kafka, follow these steps based on the context for Java and Python setups:\n\n### For Java:\nIn the project directory, run:\n```bash\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\n### For Python:\n1. **If you encounter a "Module \'kafka\' not found" error when trying to run `producer.py`:**\n   - Create a virtual environment and install the necessary packages:\n     ```bash\n     python -m venv env\n     source env/bin/activate\n     pip install -r ../requirements.txt\n     ```\n   - To activate the virtual environment:\n     ```bash\n     source env/bin/activate\n     ```\n   - To deactivate it:\n     ```bash\n     deactivate\n     ```\n   - For Windows, the activation command is:\n     ```bash\n     env/Scripts/activate\n     ```\n\n2. **For permissions issues with `build.sh`:**\n   - Run the following command in the terminal in the same directory:\n     ```bash\n     chmod +x build.sh\n     ```\n\n3. **For Docker-rela